In [43]:
import torch
import torchvision
import torch.optim as optim
import torch.nn.functional as F
import tensorboard as tb
from torch.utils.tensorboard import SummaryWriter

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import time
import pandas as pd
import json

from IPython.display import clear_output, display

In [12]:
from utils.model import Network
from utils.getData import getData
network = Network()
train_set = getData()

In [13]:
len(train_set)

60000

In [14]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple("Run", params.keys())
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

In [40]:
class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.tb = None

    def begin_run(self, run, network, loader):
        
        self.run_start_time = time.time()
        
        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')

        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)
        
        self.tb.add_image('images', grid)
        self.tb.add_graph(self.network, images)

    def end_run(self):
        self.tb.close()
        self.epoch_count=0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)

        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)

        results = OrderedDict()
        results['run'] = self.run_count
        results['epoch'] = self.epoch_count
        results['loss'] = loss
        results['accuracy'] = accuracy
        results['epoch duration'] = epoch_duration
        results['run duration'] = run_duration

        for k, v in self.run_params._asdict().items():
            results[k] = v
            
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, loss):
        self.epoch_loss += loss.item() * self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data, orient='columns'
        ).to_csv(f'{fileName}.csv')

        with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)


In [44]:
params = OrderedDict(
    lr = [0.01],
    batch_size = [1000,2000],
    shuffle= [True, False]
)

m = RunManager()

for run in RunBuilder.get_runs(params):

    network = Network()
    loader = torch.utils.data.DataLoader(train_set, batch_size=run.batch_size,shuffle=run.shuffle)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)

    m.begin_run(run, network, loader)
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            images, labels = batch
            preds = network(images)
            
            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()
            
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss)
            m.track_num_correct(preds, labels)

        m.end_epoch()
    m.end_run()
m.save('results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,0.915521,0.645400,8.279997,8.959639,0.01,1000,True
1,1,2,0.520719,0.796883,10.526356,19.543458,0.01,1000,True
2,1,3,0.437440,0.836517,11.112997,30.715182,0.01,1000,True
3,1,4,0.376752,0.861383,10.816095,41.594015,0.01,1000,True
4,1,5,0.341055,0.873800,9.930273,51.579792,0.01,1000,True
5,2,1,0.964985,0.624700,9.594634,10.208148,0.01,1000,False
6,2,2,0.527147,0.796400,9.615762,19.880469,0.01,1000,False
7,2,3,0.438594,0.839733,13.036913,32.970018,0.01,1000,False
8,2,4,0.385451,0.858217,12.512148,45.547544,0.01,1000,False
9,2,5,0.345434,0.872350,9.931485,55.544402,0.01,1000,False


## DataLoader num_workers

In [46]:
params = OrderedDict(
    lr = [0.01],
    batch_size = [1000,2000],
    shuffle= [True, False],
    num_workers = [0,1,2,4,8,16]
)

m = RunManager()

for run in RunBuilder.get_runs(params):

    network = Network()
    loader = torch.utils.data.DataLoader(train_set, batch_size=run.batch_size,shuffle=run.shuffle, num_workers=run.num_workers)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)

    m.begin_run(run, network, loader)
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            images, labels = batch
            preds = network(images)
            
            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()
            
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss)
            m.track_num_correct(preds, labels)

        m.end_epoch()
    m.end_run()
m.save('results_with_num_workers')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle,num_workers
0,1,1,0.967179,0.628517,7.909754,8.548302,0.01,1000,True,0
1,1,2,0.502497,0.809650,8.066945,16.662100,0.01,1000,True,0
2,1,3,0.418107,0.845333,8.007959,24.716920,0.01,1000,True,0
3,1,4,0.366229,0.866517,7.970418,32.734199,0.01,1000,True,0
4,1,5,0.334857,0.876333,8.036151,40.817213,0.01,1000,True,0
5,2,1,0.959330,0.629567,9.078051,14.224231,0.01,1000,True,1
6,2,2,0.501726,0.809683,8.999584,23.286309,0.01,1000,True,1
7,2,3,0.421338,0.844333,9.078113,32.442770,0.01,1000,True,1
8,2,4,0.384935,0.858883,9.078502,41.583581,0.01,1000,True,1
9,2,5,0.350005,0.870267,9.202823,50.864517,0.01,1000,True,1


KeyboardInterrupt: 